In [1]:
import random
import json
import pickle
import numpy as np
import nltk
import os
import tensorflow
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Activation,Dropout
from tensorflow.keras.optimizers import SGD


In [2]:
lemmatizer = WordNetLemmatizer()
intents = json.loads(open('test_intents.json').read())

words = []
documents = []
classes = []
ignore_words = ['?','.','!',',','_','#']

In [3]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list=nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list,intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

print(documents)

[(['start'], 'start'), (['begin'], 'start'), (['hello'], 'start'), (['hi'], 'start'), (['manual', 'locate'], 'manual_locate'), (['locate'], 'manual_locate'), (['location'], 'manual_locate'), ([], 'invalid_option'), (['{', 'location_name', '}', ',', '{', 'museum_name', '}'], 'location_and_museum'), (['book', 'a', 'ticket'], 'book_ticket'), (['ticket', 'booking'], 'book_ticket'), (['I', 'want', 'to', 'book'], 'book_ticket'), (['about', 'the', 'museum'], 'about_museum'), (['museum', 'info'], 'about_museum'), (['information', 'about', 'museum'], 'about_museum'), (['description', 'of', 'the', 'museum'], 'about_museum'), (['timings'], 'museum_timings'), (['museum', 'timings'], 'museum_timings'), (['opening', 'hours'], 'museum_timings'), (['[', 'date', ']'], 'provide_date'), (['on', '[', 'date', ']'], 'provide_date'), (['for', '[', 'date', ']'], 'provide_date'), (['I', 'want', '{', 'number_of_tickets', '}', 'tickets'], 'provide_tickets'), (['Book', '{', 'number_of_tickets', '}', 'tickets', 'f

In [4]:
words=[lemmatizer.lemmatize(word) for word in words if word not in ignore_words]
words=sorted(set(words))

classes=sorted(set(classes))
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))


In [5]:
training = []
output_empty=[0]*len(classes)

for document in documents:
    bag = []
    word_patterns=document[0]
    word_patterns=[lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(document[1])]=1
    training.append(bag+output_row)

random.shuffle(training)
training=np.array(training)

x_train=training[:,:len(words)]
y_train=training[:,len(words):]

model=tensorflow.keras.Sequential()
model.add(tensorflow.keras.layers.Dense(128,input_shape=(len(x_train[0]),),activation='relu'))
model.add(tensorflow.keras.layers.Dropout(0.5))
model.add(tensorflow.keras.layers.Dense(64,activation='relu'))
model.add(tensorflow.keras.layers.Dropout(0.5))
model.add(tensorflow.keras.layers.Dense(len(y_train[0]),activation='softmax'))
sgd=tensorflow.keras.optimizers.SGD(learning_rate=0.01,momentum=0.9,nesterov=True)
model.compile(loss='categorical_crossentropy',optimizer=sgd, metrics=['accuracy'])
hist=model.fit(x_train,y_train,epochs=200,batch_size=5,verbose=1)
model.save('chatbot_model.h5',hist)

Epoch 1/200


C:\Users\KIIT\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.0754 - loss: 2.3808
Epoch 2/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1327 - loss: 2.3205     
Epoch 3/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2422 - loss: 2.1826 
Epoch 4/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.1731 - loss: 2.1861 
Epoch 5/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1812 - loss: 2.1755     
Epoch 6/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4004 - loss: 1.9235 
Epoch 7/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3068 - loss: 1.8922     
Epoch 8/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5315 - loss: 1.8128  
Epoch 9/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4037 - loss: 1.9024 
Epoch 10/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5648 - loss: 1.5729 
Epoch 11/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5829 - loss: 1.4355 
Epoch 12/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6744 - los